###### INSTALL AMBASSADOR

In [2]:
!kubectl create clusterrolebinding my-cluster-admin-binding --clusterrole=cluster-admin --user=$(gcloud info --format="value(config.account)")!

clusterrolebinding.rbac.authorization.k8s.io/my-cluster-admin-binding created


In [3]:
!kubectl apply -f https://www.getambassador.io/yaml/ambassador/ambassador-crds.yaml

customresourcedefinition.apiextensions.k8s.io/authservices.getambassador.io created
customresourcedefinition.apiextensions.k8s.io/consulresolvers.getambassador.io created
customresourcedefinition.apiextensions.k8s.io/hosts.getambassador.io created
customresourcedefinition.apiextensions.k8s.io/kubernetesendpointresolvers.getambassador.io created
customresourcedefinition.apiextensions.k8s.io/kubernetesserviceresolvers.getambassador.io created
customresourcedefinition.apiextensions.k8s.io/logservices.getambassador.io created
customresourcedefinition.apiextensions.k8s.io/mappings.getambassador.io created
customresourcedefinition.apiextensions.k8s.io/modules.getambassador.io created
customresourcedefinition.apiextensions.k8s.io/ratelimitservices.getambassador.io created
customresourcedefinition.apiextensions.k8s.io/tcpmappings.getambassador.io created
customresourcedefinition.apiextensions.k8s.io/tlscontexts.getambassador.io created
customresourcedefinition.apiextensions.k8s.io/tracingservi

In [4]:
!kubectl apply -f https://www.getambassador.io/yaml/ambassador/ambassador-rbac.yaml

service/ambassador-admin created
clusterrole.rbac.authorization.k8s.io/ambassador created
serviceaccount/ambassador created
clusterrolebinding.rbac.authorization.k8s.io/ambassador created
deployment.apps/ambassador created


In [70]:
!kubectl apply -f k8s/ambassador/ambassador-service.yaml

service/ambassador configured


In [8]:
!kubectl apply -f k8s/ambassador/sample-deployment.yaml

deployment.apps/kubernetes-tutorial-deployment created


In [9]:
!kubectl apply -f k8s/ambassador/sample-service.yaml

service/kubernetes-tutorial-service created


##### Installing cert-manager
##### Referred to: https://auth0.com/blog/kubernetes-tutorial-managing-tls-certificates-with-ambassador/

In [10]:
! kubectl create namespace cert-manager

namespace/cert-manager created


In [11]:
! kubectl label namespace cert-manager certmanager.k8s.io/disable-validation=true

namespace/cert-manager labeled


In [50]:
!kubectl delete -n cert-manager deployment cert-manager cert-manager-cainjector cert-manager-webhook

deployment.extensions "cert-manager" deleted
deployment.extensions "cert-manager-cainjector" deleted
deployment.extensions "cert-manager-webhook" deleted


In [38]:
!kubectl apply --validate=false -f https://github.com/jetstack/cert-manager/releases/download/v0.14.1/cert-manager.yaml

customresourcedefinition.apiextensions.k8s.io/certificaterequests.cert-manager.io unchanged
customresourcedefinition.apiextensions.k8s.io/certificates.cert-manager.io unchanged
customresourcedefinition.apiextensions.k8s.io/challenges.acme.cert-manager.io unchanged
customresourcedefinition.apiextensions.k8s.io/clusterissuers.cert-manager.io unchanged
customresourcedefinition.apiextensions.k8s.io/issuers.cert-manager.io unchanged
customresourcedefinition.apiextensions.k8s.io/orders.acme.cert-manager.io unchanged
namespace/cert-manager unchanged
serviceaccount/cert-manager-cainjector unchanged
serviceaccount/cert-manager unchanged
serviceaccount/cert-manager-webhook unchanged
clusterrole.rbac.authorization.k8s.io/cert-manager-cainjector unchanged
clusterrole.rbac.authorization.k8s.io/cert-manager-controller-certificates unchanged
clusterrole.rbac.authorization.k8s.io/cert-manager-controller-issuers unchanged
clusterrole.rbac.authorization.k8s.io/cert-manager-view unchanged
clusterrole.rba

In [51]:
!kubectl apply -f https://github.com/jetstack/cert-manager/releases/download/v0.11.1/cert-manager.yaml

customresourcedefinition.apiextensions.k8s.io/challenges.acme.cert-manager.io configured
customresourcedefinition.apiextensions.k8s.io/orders.acme.cert-manager.io configured
customresourcedefinition.apiextensions.k8s.io/certificaterequests.cert-manager.io configured
customresourcedefinition.apiextensions.k8s.io/certificates.cert-manager.io configured
customresourcedefinition.apiextensions.k8s.io/clusterissuers.cert-manager.io configured
customresourcedefinition.apiextensions.k8s.io/issuers.cert-manager.io configured
namespace/cert-manager unchanged
serviceaccount/cert-manager-cainjector unchanged
serviceaccount/cert-manager unchanged
serviceaccount/cert-manager-webhook unchanged
clusterrole.rbac.authorization.k8s.io/cert-manager-cainjector unchanged
clusterrolebinding.rbac.authorization.k8s.io/cert-manager-cainjector unchanged
role.rbac.authorization.k8s.io/cert-manager-cainjector:leaderelection unchanged
rolebinding.rbac.authorization.k8s.io/cert-manager-cainjector:leaderelection conf

In [52]:
!kubectl apply -f k8s/ambassador/cluster-issuer.yaml

clusterissuer.cert-manager.io/letsencrypt-prod created


In [53]:
!kubectl apply -f k8s/ambassador/certificate.yaml

certificate.cert-manager.io/ambassador-certs created


In [56]:
!kubectl apply -f k8s/ambassador/acme-challenge.yaml

mapping.getambassador.io/acme-challenge-mapping created
service/acme-challenge-service created


In [62]:
!kubectl get secrets

NAME                     TYPE                                  DATA   AGE
ambassador-certs         kubernetes.io/tls                     3      5m19s
ambassador-token-cw9z6   kubernetes.io/service-account-token   3      160m
default-token-mz7xq      kubernetes.io/service-account-token   3      3h13m


In [68]:
!kubectl apply -f k8s/ambassador/host.yaml

host.getambassador.io/benubenu configured


In [63]:
! kubectl describe certificates ambassador-certs

Error from server (NotFound): certificates.certmanager.k8s.io "ambassador-certs" not found


In [34]:
!kubectl get pods -n cert-manager

NAME                                       READY   STATUS    RESTARTS   AGE
cert-manager-665f89d4d6-8ftwp              1/1     Running   0          23s
cert-manager-cainjector-78c8947f5c-96sh2   1/1     Running   0          24s
cert-manager-webhook-84f59fdf49-hnj2b      1/1     Running   0          23s


In [36]:
!kubectl logs -n cert-manager cert-manager-665f89d4d6-8ftwp  | grep acme-http-domain